In [84]:
train = pd.read_pickle('./data/common_dataprep/train_with_ei.pickle')
test = pd.read_pickle('./data/common_dataprep/test_with_ei.pickle')

In [77]:
def make_glms(train, test, base_formula = ''):
    # No Input
    formula0 = "ClaimNb ~ 1"
    formula1 = f"ClaimNb ~ {base_formula}  VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
    formula2 = f"{formula1} + VehBrand"
    formula3 = f"{formula2} + AreaGLM"

    formulas = [formula0, formula1, formula2, formula3]
    devs = []
    
    for formulaa in formulas:
        glm0 = smf.glm(formula=formulaa, data=train, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train['Exposure'])).fit()
        devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    
    assert min(devs) != devs[0], f'Issue with metrics: {dev}; random model predicts best?!'
    return min(devs)
    

In [78]:
train['VehPowerGLM']

0         5
1         5
2         6
3         7
4         7
         ..
678008    4
678009    4
678010    6
678011    4
678012    7
Name: VehPowerGLM, Length: 541832, dtype: int64

In [48]:
print(
    
    make_glms(train, test),
    make_glms(train, test, base_formula = 'EI_Density + EI_DrivAge + EI_BonusMalus1 + EI_BonusMalus2 + EI_VehAge +'),
    make_glms(train, test, base_formula = 'GDV_Area + GDV_VehAge + GDV_DrivAge + '),
    make_glms(train, test, base_formula = 'GDV_Area + GDV_VehAge + GDV_DrivAge + EI_Density + EI_DrivAge + EI_BonusMalus1 + EI_BonusMalus2 + EI_VehAge +')
)


KeyboardInterrupt



In [86]:
train_gan = pd.read_pickle('./data/gan_generated/sample_no_ei.pickle')
ss = joblib.load('data/gan_dataprep/scaler.pickle')
dftemp = pd.read_pickle('data/gan_dataprep/train_gan.pickle')
train_gan = pd.DataFrame(ss.inverse_transform(train_gan), columns= dftemp.columns)
train_gan = back_from_dummies(train_gan)
train_gan.head()

,ClaimNb,VehBrand,VehGas,Region,Area,VehPower,VehAge,DrivAge,DensityGLM,BonusMalus,Exposure
0,0,B6,Diesel,R93,C,5.21,8.75,46.24,5.40,50.00,0.27
1,0,B2,Diesel,R52,D,4.45,19.91,40.72,6.39,50.00,0.00
2,0,B1,Regular,R82,C,4.00,7.54,35.18,6.72,50.00,1.00
3,0,B1,Regular,R24,D,4.00,6.65,21.49,7.40,133.41,1.00
4,1,B6,Regular,R52,E,6.20,0.00,58.30,10.20,50.00,0.64


In [89]:
train_gan = prepare_gandata_for_regression(train_gan)

In [90]:
cols = ['ClaimNb', 'VehBrand', 'VehGas', 'Region', 'Area', 'VehPower', 'VehAge',
       'DrivAge', 'Density', 'BonusMalus', 'Exposure', 'DensityGLM',
       'BonusMalusGLM', 'AreaGLM', 'VehPowerGLM', 'VehAgeGLM', 'DrivAgeGLM']
joined = pd.concat([train_gan, train])[cols]

In [91]:
# No Input
formula0 = "ClaimNb ~ 1"
formula1 = f"ClaimNb ~   VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
formula2 = f"{formula1} + VehBrand"
formula3 = f"{formula2} + AreaGLM"



formulas = [formula0, formula1, formula2, formula3]
devs = []

for formulaa in formulas:
    glm0 = smf.glm(formula=formulaa, data=train_gan, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train_gan['Exposure'])).fit()
    devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    print(devs)

[32.91632850254276]
[32.91632850254276, 59.74845410521347]
[32.91632850254276, 59.74845410521347, 59.87506494269026]


TypeError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Object with dtype category cannot perform the numpy op isnan

In [ ]:
Bootstrap(9, n_bootstraps=3, n_train=5, n_test=4, random_state=0)